<a href="https://colab.research.google.com/github/ARAVINDH-20/data-science/blob/main/Settyl_Data_Science_And_Machine_Learning_Engineer_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import json, requests

# Load dataset
url = "https://gist.githubusercontent.com/farhaan-settyl/ecf9c1e7ab7374f18e4400b7a3d2a161/raw/f94652f217eeca83e36dab9d08727caf79ebdecf/dataset.json"
res = requests.get(url)
data = json.loads(res.text)
print(data)

[{'externalStatus': 'PORT OUT', 'internalStatus': 'Port Out'}, {'externalStatus': 'TERMINAL IN', 'internalStatus': 'Inbound Terminal'}, {'externalStatus': 'PORT IN', 'internalStatus': 'Port In'}, {'externalStatus': 'Vessel departure from first POL (Vessel name : TIAN FU HE)', 'internalStatus': 'Departure'}, {'externalStatus': 'Vessel arrival at final POD (Vessel name : TIAN FU HE)', 'internalStatus': 'Arrival'}, {'externalStatus': 'DEPARTCU', 'internalStatus': 'Departure'}, {'externalStatus': 'Gate in', 'internalStatus': 'Gate In'}, {'externalStatus': 'Load on MAERSK SEMBAWANG / 237E', 'internalStatus': 'Loaded on Vessel'}, {'externalStatus': 'Discharge', 'internalStatus': 'Departure'}, {'externalStatus': 'Gate out', 'internalStatus': 'Gate Out'}, {'externalStatus': 'Gate in', 'internalStatus': 'Gate In'}, {'externalStatus': 'Load on MSC SVEVA / 248E', 'internalStatus': 'Loaded on Vessel'}, {'externalStatus': 'Discharge', 'internalStatus': 'Departure'}, {'externalStatus': 'Gate out', '

In [12]:
df = pd.DataFrame(data)
df.head()

,externalStatus,internalStatus
0,PORT OUT,Port Out
1,TERMINAL IN,Inbound Terminal
2,PORT IN,Port In
3,Vessel departure from first POL (Vessel name :...,Departure
4,Vessel arrival at final POD (Vessel name : TIA...,Arrival


In [13]:
df['internalStatus'].value_counts()


internalStatus
Loaded on Vessel            331
Departure                   287
Gate Out                    146
Gate In                     143
Arrival                      62
Empty Return                 47
Empty Container Released     47
Unloaded on Vessel           37
On Rail                      25
Off Rail                     25
Outbound Terminal            24
Port Out                     15
Port In                      14
In-transit                   10
Inbound Terminal              9
Name: count, dtype: int64

In [14]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Data preprocessing
df['externalStatus'] = df['externalStatus'].str.lower()

# Tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['externalStatus'])
X = tokenizer.texts_to_sequences(df['externalStatus'])
X = pad_sequences(X)

# Label encoding: Encode target labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['internalStatus'])

# Model definition
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Dropout regularization
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

# Model compilation
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Model training
model.fit(X, y, epochs=20, batch_size=32, validation_split=0.2)  # Use validation split to monitor overfitting



Epoch 1/20
31/31 [==============================] - 1s 12ms/step - loss: 2.6198 - accuracy: 0.2712 - val_loss: 2.5033 - val_accuracy: 0.5633
Epoch 2/20
31/31 [==============================] - 0s 4ms/step - loss: 2.3041 - accuracy: 0.3613 - val_loss: 2.1412 - val_accuracy: 0.5633
Epoch 3/20
31/31 [==============================] - 0s 5ms/step - loss: 1.9977 - accuracy: 0.3838 - val_loss: 1.9088 - val_accuracy: 0.5510
Epoch 4/20
31/31 [==============================] - 0s 4ms/step - loss: 1.7347 - accuracy: 0.4831 - val_loss: 1.6098 - val_accuracy: 0.6245
Epoch 5/20
31/31 [==============================] - 0s 4ms/step - loss: 1.4831 - accuracy: 0.5650 - val_loss: 1.3543 - val_accuracy: 0.6857
Epoch 6/20
31/31 [==============================] - 0s 4ms/step - loss: 1.2680 - accuracy: 0.6274 - val_loss: 1.1917 - val_accuracy: 0.6857
Epoch 7/20
31/31 [==============================] - 0s 13ms/step - loss: 1.1174 - accuracy: 0.6735 - val_loss: 1.0672 - val_accuracy: 0.6857
Epoch 8/20
31/31 [

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
import numpy as np

# Evaluate the model
y_pred = model.predict(X)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate metrics
accuracy = accuracy_score(y, y_pred_classes)
precision = precision_score(y, y_pred_classes, average='weighted')
recall = recall_score(y, y_pred_classes, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)


39/39 [==============================] - 0s 3ms/step
Accuracy: 0.9819967266775778
Precision: 0.9774069718360858
Recall: 0.9819967266775778


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
# Evaluate model on the entire dataset
loss, accuracy = model.evaluate(X, y)

print("Overall Accuracy:", accuracy)


39/39 [==============================] - 0s 4ms/step - loss: 0.1533 - accuracy: 0.9820
Overall Accuracy: 0.9819967150688171


In [18]:

# Function to preprocess input text and make predictions
def predict_internal_status(input_text):
    # Tokenize input text
    input_sequence = tokenizer.texts_to_sequences([input_text])
    input_sequence = pad_sequences(input_sequence, maxlen=X.shape[1])

    # Make prediction
    predicted_probabilities = model.predict(input_sequence)
    predicted_index = np.argmax(predicted_probabilities, axis=1)[0]
    predicted_label = label_encoder.classes_[predicted_index]

    return predicted_label

# Example usage:
input_text = input("Enter a External Status: ").lower()   #TERMINAL IN
predicted_internal_status = predict_internal_status(input_text)
print("Predicted internal status:", predicted_internal_status)


Enter a External Status: TERMINAL IN
1/1 [==============================] - 0s 24ms/step
Predicted internal status: Gate In


In [19]:
from tensorflow.keras.models import save_model, load_model

# Save the model
model.save('settyl_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
# Load the model
loaded_model = load_model('settyl_model.h5')


In [21]:
import numpy as np
# Function to preprocess input text and make predictions
def predict_internal_status(input_text):
    # Tokenize input text
    input_sequence = tokenizer.texts_to_sequences([input_text])
    input_sequence = pad_sequences(input_sequence, maxlen=X.shape[1])

    # Make prediction
    predicted_probabilities = loaded_model.predict(input_sequence)
    predicted_index = np.argmax(predicted_probabilities, axis=1)[0]
    predicted_label = label_encoder.classes_[predicted_index]

    return predicted_label

# Example usage:
input_text = input("Enter a External Status: ").lower()   #Vessel departure from first POL
predicted_internal_status = predict_internal_status(input_text)
print("Predicted internal status:", predicted_internal_status)


Enter a External Status: Vessel departure from first POL
1/1 [==============================] - 0s 71ms/step
Predicted internal status: Departure


In [22]:
from joblib import dump, load

dump(tokenizer, "tokenizer.joblib")
dump(label_encoder, "label_encoder.joblib")

['label_encoder.joblib']

In [24]:
def predict_internal_status(input_text):
    # Load tokenizer and label encoder
    tokenizer = load("tokenizer.joblib")
    label_encoder = load("label_encoder.joblib")

    # Load the model
    loaded_model = load_model('settyl_model.h5')


    # Tokenize input text
    input_sequence = tokenizer.texts_to_sequences([input_text])
    input_sequence = pad_sequences(input_sequence, maxlen=X.shape[1])

    # Make prediction
    predicted_probabilities = loaded_model.predict(input_sequence)
    predicted_index = np.argmax(predicted_probabilities, axis=1)[0]
    predicted_label = label_encoder.classes_[predicted_index]

    return predicted_label

In [25]:
print(predict_internal_status("Terminal out"))

1/1 [==============================] - 0s 110ms/step
Port Out


In [ ]:
!pip install uvicorn fastapi nest_asyncio
!pip install pyngrok

In [8]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn

In [ ]:
app = FastAPI()

@app.get("/")
async def read_root():
    return {"message": "Hello World"}

ngrok.set_auth_token("2ep5qZ8q0L4mdoVVF0HGqHYbyVY_7kFfxewzHQZFDTFPh8RLW")

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)

nest_asyncio.apply()
uvicorn.run(app, port=8000)


In [ ]:
from fastapi import FastAPI, Query
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Define FastAPI app
app = FastAPI()

# Define input data model
class InputData(BaseModel):
    user_input: str

@app.post("/predict’")
async def get_input(data: InputData):
    data = data.dict()
    input = data['user_input']

    # Load tokenizer and label encoder
    tokenizer = load("tokenizer.joblib")
    label_encoder = load("label_encoder.joblib")

    # Load the model
    loaded_model = load_model('settyl_model.h5')


    # Tokenize input text
    input_sequence = tokenizer.texts_to_sequences([input])
    input_sequence = pad_sequences(input_sequence, maxlen=X.shape[1])

    # Make prediction
    predicted_probabilities = loaded_model.predict(input_sequence)
    predicted_index = np.argmax(predicted_probabilities, axis=1)[0]
    predicted_label = label_encoder.classes_[predicted_index]

    return {predicted_label}

# Run FastAPI server with ngrok tunneling
def run_server():
    ngrok.set_auth_token("2ep5qZ8q0L4mdoVVF0HGqHYbyVY_7kFfxewzHQZFDTFPh8RLW")
    ngrok_tunnel = ngrok.connect(8000)
    print('Public URL:', ngrok_tunnel.public_url)
    uvicorn.run(app, host="127.0.0.1", port=8000)

if __name__ == "__main__":
    nest_asyncio.apply()
    run_server()


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-61' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-35-7168d4b7d48f>", line 46, in <cell line: 44>
    run_server()
  File "<ipython-input-35-7168d4b7d48f>", line 42, in run_server
    uvicorn.run(app, host="127.0.0.1", port=8000)
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 575, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/pyt

Public URL: https://a9a7-34-75-205-197.ngrok-free.app


INFO:     Started server process [166]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2409:408d:4e03:fc9a:30b1:5afc:23b4:9d34:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2409:408d:4e03:fc9a:30b1:5afc:23b4:9d34:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2409:408d:4e03:fc9a:30b1:5afc:23b4:9d34:0 - "GET /predict HTTP/1.1" 404 Not Found
INFO:     2409:408d:4e03:fc9a:30b1:5afc:23b4:9d34:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2409:408d:4e03:fc9a:30b1:5afc:23b4:9d34:0 - "GET /openapi.json HTTP/1.1" 200 OK


1/1 [==============================] - 0s 76ms/step
INFO:     2409:408d:4e03:fc9a:30b1:5afc:23b4:9d34:0 - "POST /predict%E2%80%99 HTTP/1.1" 200 OK
